In [ ]:
# GPU
# !python3 -m pip install paddlepaddle-gpu==2.6.1.post120 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html

# CPU
!pip install paddlepaddle

In [ ]:
!git clone https://github.com/PaddlePaddle/PaddleOCR
!pip3 install -r /content/PaddleOCR/requirements.txt
!pip install paddleocr

fatal: destination path 'PaddleOCR' already exists and is not an empty directory.


In [ ]:
import paddleocr
# ocr = paddleocr.PaddleOCR(lang="en")
ocr = paddleocr.PaddleOCR(use_angle_cls=True, lang="en")

[2024/09/15 05:11:10] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c

In [ ]:
import re
import os
import requests
import pandas as pd
import multiprocessing
import time
from time import time as timer
from tqdm import tqdm
import numpy as np
from pathlib import Path
from functools import partial
import requests
import urllib
from PIL import Image
from io import BytesIO


def create_placeholder_image(image_save_path):
    try:
        placeholder_image = Image.new('RGB', (100, 100), color='black')
        placeholder_image.save(image_save_path)
    except Exception as e:
        return

def download_image(image_link, save_folder, retries=3, delay=3):
    if not isinstance(image_link, str):
        return

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    if os.path.exists(image_save_path):
        return

    for _ in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            return
        except:
            time.sleep(delay)

    create_placeholder_image(image_save_path) #Create a black placeholder image for invalid links/images

def download_images(image_links, download_folder, allow_multiprocessing=True):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    if allow_multiprocessing:
        download_image_partial = partial(download_image, save_folder=download_folder, retries=3, delay=3)

        with multiprocessing.Pool(64) as pool:
            list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
            pool.close()
            pool.join()
    else:
        for image_link in tqdm(image_links, total=len(image_links)):
            download_image(image_link, save_folder=download_folder, retries=3, delay=3)


def getImageByLink(image_link):
    response = requests.get(image_link)
    img = Image.open(BytesIO(response.content))

    # Convert the image to a format that PaddleOCR can process
    img = img.convert('RGB')
    img = np.array(img)

    return img

In [ ]:
import os
import pandas as pd

def get_text_from_image(image_link):
    try:
        img = getImageByLink(image_link)
        result = ocr.ocr(img)

        if result is not None and len(result) > 0 and result[0] is not None:
            texts = [res[1][0] for res in result[0]]
            extracted_text = "--".join(texts)
        else:
            extracted_text = "No text extracted"
    except Exception as e:
        extracted_text = f"Error during extraction: {str(e)}"

    return extracted_text

def get_text_from_img(df, save_interval=500, save_path='/content/test_text_5.csv'):
    df['extracted_text'] = ""

    for i in range(len(df)):
        try:
            img = getImageByLink(df['image_link'][i])
            result = ocr.ocr(img)

            if result is not None and len(result) > 0 and result[0] is not None:
                texts = [res[1][0] for res in result[0]]
                extractedContext = "--".join(texts)
            else:
                extractedContext = "No text extracted"

            df['extracted_text'][i] = extractedContext
        except Exception as e:
            df['extracted_text'][i] = f"Error during extraction: {str(e)}"

        # Save the DataFrame every `save_interval` iterations
        if (i + 1) % save_interval == 0:
            df.to_csv(save_path, index=False)
            print(f"Saved progress at {i + 1} images")

    # Final save to ensure the last batch is saved
    df.to_csv(save_path, index=False)
    print("Final save completed")

    return df

# Example usage
df = pd.read_csv('/content/train.csv', encoding='latin-1')
df1 = df[500:2000].copy()
df1.reset_index(drop=True, inplace=True)

df2 = get_text_from_img(df1, save_interval=50, save_path='./drive/MyDrive/AmazonML/train_text_2.csv')


[2024/09/15 05:11:15] ppocr DEBUG: dt_boxes num : 7, elapsed : 0.9518558979034424
[2024/09/15 05:11:15] ppocr DEBUG: cls num  : 7, elapsed : 0.2504730224609375
[2024/09/15 05:11:16] ppocr DEBUG: rec_res num  : 7, elapsed : 1.3592002391815186
[2024/09/15 05:11:18] ppocr DEBUG: dt_boxes num : 19, elapsed : 1.50803542137146
[2024/09/15 05:11:18] ppocr DEBUG: cls num  : 19, elapsed : 0.11565089225769043
[2024/09/15 05:11:25] ppocr DEBUG: rec_res num  : 19, elapsed : 6.653963565826416
[2024/09/15 05:11:25] ppocr DEBUG: dt_boxes num : 9, elapsed : 0.30462050437927246
[2024/09/15 05:11:25] ppocr DEBUG: cls num  : 9, elapsed : 0.045322418212890625
[2024/09/15 05:11:26] ppocr DEBUG: rec_res num  : 9, elapsed : 0.8481783866882324
[2024/09/15 05:11:27] ppocr DEBUG: dt_boxes num : 4, elapsed : 0.45592355728149414
[2024/09/15 05:11:27] ppocr DEBUG: cls num  : 4, elapsed : 0.05143570899963379
[2024/09/15 05:11:28] ppocr DEBUG: rec_res num  : 4, elapsed : 0.5858073234558105
[2024/09/15 05:11:28] ppoc

In [ ]:
import pandas as pd

df2= pd.read_csv("./drive/MyDrive/AmazonML/train_text_1.csv")
df3= pd.read_csv("./drive/MyDrive/AmazonML/train_text_2.csv")
df4= pd.concat([df2, df3])
df4.shape
# df3.tail()

# df3.to_csv("./drive/MyDrive/AmazonML/train_text_2.csv", index=False)
df4.to_csv("./drive/MyDrive/AmazonML/train_text_3.csv", index=False)

In [ ]:
df2.to_csv("./drive/MyDrive/AmazonML/train_text_2.csv", index=False)
df2.to_csv("/content/train_text_2.csv", index=False)
print(df2.shape)